In [1]:
from collections import OrderedDict
from Portable import osp

import json
import re

解析分片合并

In [ ]:
parse_dir = r"./parsed"
result_file = r"./result/parsed.json"

In [ ]:
merge_dict = OrderedDict()
files = sorted(osp.als(parse_dir))
for parse_file in files:
    with open(parse_file, "r", encoding="utf8") as reader:
        data = json.load(reader)
    key = osp.refmt(osp.op_base(parse_file), ".jpg")
    merge_dict[key] = data

In [ ]:
with open(result_file, "w", encoding="utf8") as writer:
    json.dump(merge_dict, writer, indent=4, ensure_ascii=False)

方位问题抽取

In [2]:
source = r"./result/parsed_rect.json"
output = r"./result/direction_sort/directions_only.json"
rests = r"./result/direction_sort/rests.json"
ques_out = r"./result/direction_sort/ques_sort.json"
res_ques = r"./result/direction_sort/rest_ques.json"

osp.ensure_dirs(output, is_file=True)

'./result/direction_sort/directions_only.json'

In [3]:
with open(source, "r", encoding="utf8") as reader:
    data = json.load(reader)

contains = [r".{1,2}方", r".{1,2}侧", r".边", r".面", r".手边", "左前", "左后", "右前", "右后"]
over_int = ["前", "后", "左", "右"]
def pred_options(op:str) -> bool:
    if not isinstance(op, str): return False
    result = any([len(re.findall(ct, op)) > 0 for ct in contains])
    result = result or any([oi == op for oi in over_int])
    return result

direction_dict, rests_dict = dict(), dict()
counter = [0, 0]
sorted_ques, rest_ques = list(), list()
for k, v in data.items():
    direction_flag = False
    for e in v:
        opt_options = e["question"].split("？")
        sing_flag = len(opt_options) == 2 and "方位" in opt_options[1]
        direction_flag = direction_flag or sing_flag
        parse_options = e["parse"]["options"]
        sing_flag = parse_options and any([pred_options(po) for po in parse_options])
        direction_flag = direction_flag or sing_flag
        counter[0 if sing_flag else 1] += 1
        if sing_flag: sorted_ques.append([k, e["question"]])
        else: rest_ques.append([k, e["question"]])
    if direction_flag: direction_dict.update({k: v})
    else: rests_dict.update({k: v})

with open(output, "w", encoding="utf8") as out_writer,\
    open(rests, "w", encoding="utf8") as res_writer,\
    open(ques_out, "w", encoding="utf8") as ques_writer,\
    open(res_ques, "w", encoding="utf8") as req_writer:
    json.dump(direction_dict, out_writer, indent=4, ensure_ascii=False)
    json.dump(rests_dict, res_writer, indent=4, ensure_ascii=False)
    json.dump(sorted_ques, ques_writer, indent=4, ensure_ascii=False)
    json.dump(rest_ques, req_writer, indent=4, ensure_ascii=False)

In [4]:
print(f"sorting results: \nsort<{len(direction_dict)}>\nrest<{len(rests_dict)}>")
print(f"\nquestion layer: \nsort<{counter[0]}>\nrest<{counter[1]}>")

sorting results: 
sort<1524>
rest<410>

question layer: 
sort<3897>
rest<2652>


In [ ]:
# print(f"ques_sorted_ovl: {len(sorted_ques)} =?= {len(set(sorted_ques))}")
# print(f"ques_rest_ovl:   {len(rest_ques)} =?= {len(set(rest_ques))}")